In [1]:
import numpy as np
import pandas as pd
import itertools
import random

# importing parameters

In [2]:
ip = pd.read_excel("parameters.xlsx")
line_currents = ip.line_currents  # Convert to numpy array
line_impedance = ip.line_impedence
for i in range(len(line_impedance )):
    line_impedance [i]=(line_impedance [i]/1.602756)
line_currents=line_currents.tolist()
line_impedance=line_impedance.tolist()
print(line_impedance )
print(line_currents)

[0.05752591161723931, 0.3075951673242839, 0.22835665566062455, 0.23777792751984705, 0.5109948114372992, 0.11679881404281126, 1.0677857390644614, 0.642643047350938, 0.6513780013926013, 0.12266371175649943, 0.2335976280856225, 0.9159223237972591, 0.33791793635462913, 0.36873984561592654, 0.4656354429495194, 0.8042396971217077, 0.4567133113212491, 0.10232374734519789, 0.9385084192478456, 0.2554974057186496, 0.4423006371525048, 0.28151509025703225, 0.5602849092438275, 0.559037058666447, 0.12665683360411692, 0.17731956704576368, 0.6607368807229547, 0.5017607171646838, 0.3166420840102922, 0.6079528012997611, 0.19372880213831675, 0.2127585234433688]
[0.0459509521120074, 0.0408333197278888, 0.029329336848964, 0.0278878109574774, 0.0278878109574774, 0.0127475487839819, 0.0103788245962632, 0.00805294977011529, 0.00733484832835689, 0.00670820393249937, 0.00608358446970205, 0.00541294744108974, 0.00460977222864644, 0.00306757233003559, 0.00246981780704569, 0.00174642491965729, 0.0010770329614269, 

In [3]:
ev_power_ratings = 56.25  # EVCS power rating in MW
voltage_level = 12.66  # Voltage level in kV
load_current = (ev_power_ratings/voltage_level )/7898 # I = P / V in (per units)

# data processing

In [4]:
def calculate_power_loss(current, impedance):
    return (current**2)*impedance

In [5]:
def total_power_loss(evcs_locations, line_currents, line_impedance, load_current):
    adjusted_currents = line_currents.copy()
    for evcs in evcs_locations:
        adjusted_currents[evcs-1] += load_current
    total_loss = 0
    for i in range(len(adjusted_currents)):
        total_loss += calculate_power_loss(adjusted_currents[i], line_impedance[i])
    return total_loss

In [6]:
sectors = {
    'sector_1': list(range(8, 19)),
    'sector_2': list(range(19, 23)),
    'sector_3':list(range(23, 26)),
    'sector_4': list(range(26, 33))
}

possible_locations = {
    'sector_1': sectors['sector_1'],
    'sector_2': sectors['sector_2'],
    'sector_3': sectors['sector_3'],
    'sector_4': sectors['sector_4']
}
random.shuffle(possible_locations['sector_1'])
random.shuffle(possible_locations['sector_2'])
random.shuffle(possible_locations['sector_3'])
random.shuffle(possible_locations['sector_4'])
print(possible_locations)


{'sector_1': [17, 13, 11, 9, 10, 16, 15, 18, 12, 14, 8], 'sector_2': [22, 21, 19, 20], 'sector_3': [23, 24, 25], 'sector_4': [28, 26, 29, 32, 27, 30, 31]}


In [7]:
data = []
for loc1 ,loc2,loc3,loc4 in list(itertools.product(possible_locations['sector_1'],possible_locations['sector_2'],possible_locations['sector_3'],possible_locations['sector_4'])):
    current_locations = [loc1, loc2, loc3, loc4]
    loss = total_power_loss(current_locations, line_currents, line_impedance, load_current)
    data.append((*current_locations, loss))

> making processed data to csv file

In [8]:
df = pd.DataFrame(data, columns=['loc1', 'loc2', 'loc3', 'loc4', 'loss'])
df.to_csv("evcs_power_loss_data.csv", index = False)
print(df)

     loc1  loc2  loc3  loc4      loss
0      17    22    23    28  0.002110
1      17    22    23    26  0.002106
2      17    22    23    29  0.002107
3      17    22    23    32  0.002103
4      17    22    23    27  0.002113
..    ...   ...   ...   ...       ...
919     8    20    25    29  0.002105
920     8    20    25    32  0.002102
921     8    20    25    27  0.002111
922     8    20    25    30  0.002105
923     8    20    25    31  0.002102

[924 rows x 5 columns]


# random forest model

In [9]:
from sklearn.model_selection import train_test_split ,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [10]:
df = pd.read_csv('evcs_power_loss_data.csv')

In [11]:
X = df[['loc1', 'loc2', 'loc3', 'loc4']]
y = df['loss']

In [12]:
param_grid = {
    'n_estimators': [100, 200, 300], 
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4] 
}

In [13]:
rf = RandomForestRegressor(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, verbose=2)

In [14]:
rf_best = rf_random.best_estimator_

# training and testing method

In [15]:
run= 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=run)
rf_best.fit(X_train, y_train)
y_pred = rf_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error for run :", mse)

Mean Squared Error for run : 4.360047858554113e-13


In [16]:
new_possible_locations = list(itertools.product(possible_locations['sector_1'], possible_locations['sector_2'], possible_locations['sector_3'], possible_locations['sector_4']))
new_predictions = rf_best.predict(new_possible_locations)
min_loss_idx = np.argmin(new_predictions)
optimal_locations = new_possible_locations[min_loss_idx]
min_power_loss = new_predictions[min_loss_idx]

print("the optimal locations for EVCS are:", optimal_locations, "with a predicted total power loss of", min_power_loss,"in (pu's)", min_power_loss*100000,"in KW")
print()

the optimal locations for EVCS are: (18, 21, 25, 32) with a predicted total power loss of 0.0020964341085974634 in (pu's) 209.64341085974633 in KW



C:\Users\bhime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
